In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb

from datetime import datetime
from sklearn.metrics import mean_absolute_error
from sklearn.cross_validation import KFold
from scipy.stats import skew, boxcox
from sklearn.preprocessing import StandardScaler
from pylightgbm.models import GBMRegressor
import itertools
import os
os.environ['LIGHTGBM_EXEC'] = "/Users/shashankbhushan/Documents/Github/LightGBM/lightgbm"
%matplotlib inline

/Users/shashankbhushan/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
## Model check
def mae_cv(model, X, Y):
    rmse = cross_val_score(model, X, Y,scoring="neg_mean_absolute_error", cv=5)
    return rmse

def evalerror(preds, dtrain):
    labels = dtrain.get_label()
    return 'mae', mean_absolute_error(np.exp(preds), np.exp(labels))

def logregobj(preds, dtrain):
    labels = dtrain.get_label()
    con = 2
    x = preds-labels
    grad = con*x / (np.abs(x)+con)
    hess = con**2 / (np.abs(x)+con)**2
    return grad, hess

def split_cont6(val):
    if(val >= 0.158):
        return 1
    else:
        return 0

def encode(charcode):
    r = 0
    ln = len(str(charcode))
    for i in range(ln):
        r += (ord(str(charcode)[i]) - ord('A') + 1) * 26 ** (ln - i - 1)
    return r

fair_constant = 2
def fair_obj(preds, dtrain):
    labels = dtrain.get_label()
    x = (preds - labels)
    den = abs(x) + fair_constant
    grad = fair_constant * x / (den)
    hess = fair_constant * fair_constant / (den * den)
    return grad, hess

def xg_eval_mae(yhat, dtrain):
    y = dtrain.get_label()
    return 'mae', mean_absolute_error(np.exp(y)-shift,
                                      np.exp(yhat)-shift)

def mungeskewed(train, test, numeric_feats):
    ntrain = train.shape[0]
    test['loss'] = 0
    train_test = pd.concat((train, test)).reset_index(drop=True)
    train_test['cont6_split'] = train_test['cont6'].apply(split_cont6)
    skewed_feats = train[numeric_feats].apply(lambda x: skew(x.dropna()))
    skewed_feats = skewed_feats[skewed_feats > 0.25]
    skewed_feats = skewed_feats.index

    for feats in skewed_feats:
        train_test[feats] = train_test[feats] + 1
        train_test[feats], lam = boxcox(train_test[feats])
    return train_test, ntrain

shift = 200
COMB_FEATURE = 'cat80,cat87,cat57,cat12,cat79,cat10,cat7,cat89,cat2,cat72,' \
               'cat81,cat11,cat1,cat13,cat9,cat3,cat16,cat90,cat23,cat36,' \
               'cat73,cat103,cat40,cat28,cat111,cat6,cat76,cat50,cat5,' \
               'cat4,cat14,cat38,cat24,cat82,cat25'.split(',')

In [3]:
train = pd.read_csv('Data/train.csv')
test = pd.read_csv('Data/test.csv')

test['loss'] = np.nan
joined = pd.concat((train, test)).reset_index(drop=True)

rows = joined.shape[0];
for column in list(train.select_dtypes(include=['object']).columns):
    if train[column].nunique() != test[column].nunique():
        set_train = set(train[column].unique())
        set_test = set(test[column].unique())
        remove_train = set_train - set_test
        remove_test = set_test - set_train

        remove = remove_train.union(remove_test)
        def filter_cat(x):
            if x in remove:
                return np.nan
            return x

        joined[column] = joined[column].apply(lambda x: filter_cat(x), 1)
    ## Should have worked but is not working
    '''
    temp = joined.groupby(column).count()['id']
    mapObj = {}
    data = temp.values.astype(float)/rows
    ind = 0
    for d in data:
        if d < .02:
            mapObj[temp.index[ind]] = 'Other'
        else:
            mapObj[temp.index[ind]] = temp.index[ind]
        ind+=1
    joined[column] = joined[column].map(mapObj)
    '''
    #joined[column] = pd.factorize(joined[column].values, sort=True)[0]

# there is some information gain
#joined['cont6_split'] = joined['cont6'].apply(split_cont6)

numeric_feats = [x for x in train.columns[1:-1] if 'cont' in x]

joined, ntrain = mungeskewed(joined[joined['loss'].notnull()].copy(), joined[joined['loss'].isnull()].copy(), numeric_feats)
categorical_feats = [x for x in train.columns[1:-1] if 'cat' in x]

print('')
for comb in itertools.combinations(COMB_FEATURE, 2):
    feat = comb[0] + "_" + comb[1]
    joined[feat] = joined[comb[0]] + joined[comb[1]]
    joined[feat] = pd.factorize(joined[feat], sort=True)[0]
    print('Analyzing Columns:', feat)

#categorical_feats = [x for x in train_test.columns[1:] if 'cat' in x]

print('')
for col in categorical_feats:
    print('Analyzing Column:', col)
    joined[col] = pd.factorize(joined[col], sort=True)[0]


('Analyzing Columns:', 'cat80_cat87')
('Analyzing Columns:', 'cat80_cat57')
('Analyzing Columns:', 'cat80_cat12')
('Analyzing Columns:', 'cat80_cat79')
('Analyzing Columns:', 'cat80_cat10')
('Analyzing Columns:', 'cat80_cat7')
('Analyzing Columns:', 'cat80_cat89')
('Analyzing Columns:', 'cat80_cat2')
('Analyzing Columns:', 'cat80_cat72')
('Analyzing Columns:', 'cat80_cat81')
('Analyzing Columns:', 'cat80_cat11')
('Analyzing Columns:', 'cat80_cat1')
('Analyzing Columns:', 'cat80_cat13')
('Analyzing Columns:', 'cat80_cat9')
('Analyzing Columns:', 'cat80_cat3')
('Analyzing Columns:', 'cat80_cat16')
('Analyzing Columns:', 'cat80_cat90')
('Analyzing Columns:', 'cat80_cat23')
('Analyzing Columns:', 'cat80_cat36')
('Analyzing Columns:', 'cat80_cat73')
('Analyzing Columns:', 'cat80_cat103')
('Analyzing Columns:', 'cat80_cat40')
('Analyzing Columns:', 'cat80_cat28')
('Analyzing Columns:', 'cat80_cat111')
('Analyzing Columns:', 'cat80_cat6')
('Analyzing Columns:', 'cat80_cat76')
('Analyzing Col

In [4]:
ss = StandardScaler()
joined[numeric_feats] = ss.fit_transform(joined[numeric_feats].values)

train = joined[joined['loss'].notnull()].copy()
test = joined[joined['loss'].isnull()].copy()

print('\nMedian Loss:', train.loss.median())
print('Mean Loss:', train.loss.mean())

ids = pd.read_csv('Data/test.csv')['id']

## TODO: TO run one model with the id as a feature
y = np.log(train['loss'] + shift)
ids = test['id']
X = train.drop(['loss', 'id'], 1)
X_test = test.drop(['loss', 'id'], 1)

('\nMedian Loss:', 960.98)
('Mean Loss:', 1822.398740505439)


In [6]:
n_folds = 10
cv_sum = 0
early_stopping = 100
fpred = []
xgb_rounds = []

kf = KFold(train.shape[0], n_folds=n_folds)
clf = GBMRegressor(
        num_threads=-1,
        learning_rate = 0.01,
        num_iterations = 2336,
        verbose = False,
        feature_fraction = 0.7,
        bagging_fraction = 0.7,
        max_depth = 12,
        bagging_seed = 2016,
        early_stopping_round = 50
        )

for i, (train_index, test_index) in enumerate(kf):
    print('\n Fold %d' % (i+1))
    X_train, X_val = X.iloc[train_index], X.iloc[test_index]
    y_train, y_val = y.iloc[train_index], y.iloc[test_index]

    rand_state = 2016
    
    clf.fit(X_train, y_train)
    scores_val = clf.predict(X_val)
    cv_score = mean_absolute_error(np.exp(y_val), np.exp(scores_val))
    print('eval-MAE: %.6f' % cv_score)
    y_pred = np.exp(clf.predict(X_test)) - shift
    if i > 0:
        fpred = pred + y_pred
    else:
        fpred = y_pred
        pred = fpred
        cv_sum = cv_sum + cv_score



 Fold 1
eval-MAE: 2387.080315


/Users/shashankbhushan/anaconda2/lib/python2.7/site-packages/numpy/lib/npyio.py:893: UserWarning: loadtxt: Empty input file: "/var/folders/ph/vtnbjsd908n2hg09dj0znzdh0000gn/T/tmp3soBuh/LightGBM_predict_result.txt"
  warnings.warn('loadtxt: Empty input file: "%s"' % fname)



 Fold 2


KeyboardInterrupt: 

In [32]:
mpred = pred / n_folds
score = cv_sum / n_folds
print('Average eval-MAE: %.6f' % score)

print("Writing results")
result = pd.DataFrame(mpred, columns=['loss'])
result["id"] = ids
result = result.set_index("id")
print("%d-fold average prediction:" % n_folds)

now = datetime.now()
score = str(round((cv_sum / n_folds), 6))
sub_file = 'submission_10fold-average-lightGBM_' + str(score) + '_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'
print("Writing submission: %s" % sub_file)
result.to_csv(sub_file, index=True, index_label='id')

Average eval-MAE: 114.510558
Writing results
10-fold average prediction:
Writing submission: submission_10fold-average-lightGBM_114.510558_2016-11-20-19-14.csv


## Starting with LightGBM model

In [28]:
import os
os.environ['LIGHTGBM_EXEC'] = "/Users/shashankbhushan/Documents/Github/LightGBM/lightgbm"
model = GBMRegressor(
    num_threads=-1,
    learning_rate = 0.01,
    num_iterations = int(2012 / 0.9), 
    verbose = False, 
    feature_fraction = 0.5,
    bagging_fraction = 0.8,
    max_depth = 12
)

In [16]:
X['loss'] = y
from sklearn.model_selection import train_test_split

train_local, validation = train_test_split(
    X, 
    test_size = 0.2, 
    random_state = 0
)

In [20]:
%%time 
features = np.array([column for column in train_local.columns if column != 'loss'])
model.fit(
    train_local[features].values, 
    train_local['loss'].values, 
    test_data = [(
        validation[features].values, 
        validation['loss'].values
    )]
)

CPU times: user 11.5 s, sys: 1.38 s, total: 12.9 s
Wall time: 1min 58s


In [22]:
p = np.exp(model.predict(validation[features].values)) - shift

In [23]:
scores = []
for prediction_multiplicator in np.linspace(0.95, 1.05, 101) :
    score = mean_absolute_error(p * prediction_multiplicator, validation['loss'])
    scores.append((score, prediction_multiplicator))

M = min(scores)[1]
validation.loc[:, 'loss_lightgbm'] = p * M
print 'Min error: %.2f, optimal prediction multiplicator: %.3f' % min(scores)

Min error: 2551.77, optimal prediction multiplicator: 0.950


/Users/shashankbhushan/anaconda2/lib/python2.7/site-packages/pandas/core/indexing.py:296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/shashankbhushan/anaconda2/lib/python2.7/site-packages/pandas/core/indexing.py:476: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [29]:
model.fit(X[features].values, X['loss'].values)

In [30]:
prediction = np.exp(model.predict(X_test)) - shift

submission = pd.DataFrame()
submission['loss'] = prediction*0.950
submission['id'] = ids
submission.to_csv('LibSVM.csv', index=False)

In [36]:
submission = pd.DataFrame()
submission['loss'] = prediction*0.950
submission['id'] = ids
submission.to_csv('LibSVM.csv', index=False)

In [33]:
cv_sum

1145.1055814626091